In [ ]:
# ==============================================================================
# notebooks/10_results_analysis_and_visualization.ipynb
# ==============================================================================

# # 10 - Results Analysis and Visualization
# This notebook provides a comprehensive analysis and visualization of the coffee yield prediction model's results. It aims to:
# 1.  Load the master dataset (containing actual yields) and the 2025 predictions.
# 2.  Evaluate the model's performance on historical data.
# 3.  Visualize actual vs. predicted yields to assess model fit.
# 4.  Display the 2025 predicted yields spatially.
# 5.  Revisit and visualize feature importance.
# 6.  Summarize key insights.

# ## 1. Load Project Setup and Libraries
# Import `pandas`, `numpy`, `os`, `matplotlib.pyplot`, `seaborn`, `geopandas`, and `pickle` (to load model artifacts).
# Also, import evaluation metrics from `sklearn.metrics`.

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import pickle # To load the saved model and scaler

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

print("Libraries loaded.")

# Define data directories
processed_data_dir = '../data/processed/'
model_dir = '../models/'

os.makedirs(processed_data_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

print(f"Processed data directory: {processed_data_dir}")
print(f"Models directory: {model_dir}")

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

# ## 2. Load Data and Model Artifacts
# Load the master dataset, the 2025 predictions, the original woreda boundaries (for spatial visualization),
# and the trained model/scaler/feature columns.

df_master = None
df_predictions_2025 = None
gdf_woredas = None
best_model = None
scaler = None
feature_cols = None

try:
    df_master = pd.read_csv(os.path.join(processed_data_dir, 'master_woreda_data.csv'))
    print(f"Loaded master dataset: {df_master.shape[0]} records.")

    df_predictions_2025 = pd.read_csv(os.path.join(processed_data_dir, 'sidama_coffee_yield_predictions_2025.csv'))
    print(f"Loaded 2025 predictions: {df_predictions_2025.shape[0]} records.")

    gdf_woredas = gpd.read_file(os.path.join(processed_data_dir, 'sidama_woredas.geojson'))
    print(f"Loaded woreda geometries: {len(gdf_woredas)} woredas.")

    # Load trained model artifacts
    with open(os.path.join(model_dir, 'best_yield_prediction_model.pkl'), 'rb') as f:
        best_model = pickle.load(f)
    print(f"Loaded best model: {type(best_model).__name__}")

    with open(os.path.join(model_dir, 'feature_scaler.pkl'), 'rb') as f:
        scaler = pickle.load(f)
    print("Loaded feature scaler.")

    with open(os.path.join(model_dir, 'feature_columns.pkl'), 'rb') as f:
        feature_cols = pickle.load(f)
    print(f"Loaded feature columns: {len(feature_cols)}")

except FileNotFoundError as e:
    print(f"Error loading data/model artifacts: {e}. Please ensure previous notebooks (08, 09) were run successfully.")

print("\nData and model artifacts loading complete.")

# ## 3. Model Performance Evaluation on Historical Data
# Re-evaluate the best model's performance on the historical training data to confirm metrics and visualize residuals.

if df_master is not None and best_model is not None and scaler is not None and feature_cols is not None:
    df_train_actuals = df_master.dropna(subset=['annual_yield_quintals_ha']).copy()

    X_train = df_train_actuals[feature_cols]
    y_train = df_train_actuals['annual_yield_quintals_ha']

    # Impute missing values in X_train using the same means as during training
    # (This assumes the means were calculated on X during 09_yield_prediction_model.ipynb
    # and that the scaler wasn't solely responsible for imputation. If imputation was part of scaler
    # pipeline, then this step might not be strictly needed, but it's safer).
    # Re-run the imputation part of notebook 09 if you hit errors here regarding NaNs.
    for col in feature_cols:
        if X_train[col].isnull().any():
            # For simplicity, using mean of the current X_train. In a real pipeline,
            # you'd save/load the imputer from the training phase.
            mean_val = X_train[col].mean()
            X_train[col] = X_train[col].fillna(mean_val)

    X_train_scaled = scaler.transform(X_train) # Use the loaded scaler

    y_pred_train = best_model.predict(X_train_scaled)

    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    r2_train = r2_score(y_train, y_pred_train)
    mae_train = mean_absolute_error(y_train, y_pred_train)

    print(f"\n--- Model Performance on Historical Data ({type(best_model).__name__}) ---")
    print(f"RMSE: {rmse_train:.3f}")
    print(f"R2 Score: {r2_train:.3f}")
    print(f"MAE: {mae_train:.3f}")

    # Plot Actual vs. Predicted (Historical)
    plt.figure(figsize=(10, 6))
    sns.regplot(x=y_train, y=y_pred_train, scatter_kws={'alpha':0.6}, line_kws={'color':'red'})
    plt.xlabel('Actual Yield (quintals/ha)')
    plt.ylabel('Predicted Yield (quintals/ha)')
    plt.title('Actual vs. Predicted Coffee Yields (Historical Data)')
    plt.grid(True)
    plt.show()

    # Plot Residuals
    residuals = y_train - y_pred_train
    plt.figure(figsize=(10, 6))
    sns.histplot(residuals, kde=True, bins=30)
    plt.xlabel('Residuals (Actual - Predicted)')
    plt.ylabel('Frequency')
    plt.title('Distribution of Residuals')
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.scatter(y_pred_train, residuals, alpha=0.6)
    plt.axhline(y=0, color='red', linestyle='--')
    plt.xlabel('Predicted Yield')
    plt.ylabel('Residuals')
    plt.title('Residuals vs. Predicted Values')
    plt.grid(True)
    plt.show()
else:
    print("Skipping model performance evaluation due to missing data/model artifacts.")

# ## 4. Visualize 2025 Predicted Yields Spatially
# Merge the 2025 predictions with the woreda geometries to visualize the predicted yield distribution across Sidama.

if df_predictions_2025 is not None and gdf_woredas is not None:
    gdf_predictions = gdf_woredas.merge(df_predictions_2025, left_on='Woreda_ID', right_on='woreda_id', how='left')

    print(f"\nGeoDataFrame with 2025 predictions: {gdf_predictions.shape[0]} records.")
    print(gdf_predictions.head())

    # Plotting the predicted yields
    fig, ax = plt.subplots(1, 1, figsize=(12, 10))
    gdf_predictions.plot(
        column='predicted_yield_quintals_ha',
        cmap='YlGn',
        linewidth=0.8,
        ax=ax,
        edgecolor='0.8',
        legend=True,
        legend_kwds={'label': "Predicted Coffee Yield (quintals/ha) in 2025"}
    )
    ax.set_title('2025 Predicted Coffee Yields by Woreda in Sidama, Ethiopia')
    ax.set_axis_off()
    plt.show()
else:
    print("Skipping spatial visualization due to missing prediction or woreda data.")

# ## 5. Feature Importance Analysis (Revisit)
# Visualize the feature importances to reinforce understanding of which factors most influence coffee yield predictions.

if best_model is not None and feature_cols is not None:
    print("\n--- Feature Importance/Coefficients Visualization ---")
    if hasattr(best_model, 'feature_importances_'):
        feature_importance = pd.DataFrame({
            'Feature': feature_cols,
            'Importance': best_model.feature_importances_
        }).sort_values(by='Importance', ascending=False)

        plt.figure(figsize=(12, 8))
        sns.barplot(x='Importance', y='Feature', data=feature_importance.head(15))
        plt.title('Top 15 Feature Importances')
        plt.xlabel('Importance')
        plt.ylabel('Feature')
        plt.show()

    elif hasattr(best_model, 'coef_'): # For linear models like Ridge/Lasso, or final_estimator of Stacking
        # Handle Stacking Regressor's final estimator coefficients
        if isinstance(best_model, StackingRegressor) and hasattr(best_model.final_estimator_, 'coef_'):
            coefficients = best_model.final_estimator_.coef_
            # Note: these coefficients are for the *meta-features* (outputs of base models)
            # not the original features directly.
            meta_feature_names = [f'base_model_{name}' for name, _ in best_model.estimators]
            coef_df = pd.DataFrame({
                'Feature': meta_feature_names,
                'Coefficient': coefficients
            }).sort_values(by='Coefficient', key=abs, ascending=False)
            plt.figure(figsize=(10, 6))
            sns.barplot(x='Coefficient', y='Feature', data=coef_df)
            plt.title('Stacking Regressor: Final Estimator Coefficients (Meta-Features)')
            plt.xlabel('Coefficient Value')
            plt.ylabel('Base Model / Meta-Feature')
            plt.show()

        # For simple linear models
        elif not isinstance(best_model, StackingRegressor):
            coefficients = pd.DataFrame({
                'Feature': feature_cols,
                'Coefficient': best_model.coef_
            }).sort_values(by='Coefficient', key=abs, ascending=False)

            plt.figure(figsize=(12, 8))
            sns.barplot(x='Coefficient', y='Feature', data=coefficients.head(15))
            plt.title('Top 15 Feature Coefficients')
            plt.xlabel('Coefficient Value')
            plt.ylabel('Feature')
            plt.show()

    else:
        print("Feature importances/coefficients not directly visualizable for this model type.")
else:
    print("Skipping feature importance visualization due to missing model or feature columns.")

# ## 6. Conclusion and Next Steps
# Summarize the findings and discuss potential next steps for improving the model or deploying it.

# This notebook concludes the yield prediction project by analyzing the model's performance and visualizing the results.

# **Key Insights from Analysis:**
# * **Model Performance:** [Summarize RMSE, R2, MAE values from Section 3. Discuss if these are satisfactory. E.g., "The model achieved an R2 of X, indicating that X% of the variance in coffee yield can be explained by the features."]
# * **Residuals:** [Describe the residual plots. Are they randomly distributed around zero? Is there any heteroscedasticity or pattern? E.g., "Residuals appear mostly homoscedastic and centered around zero, suggesting a reasonable model fit, though some outliers are present."]
# * **Predicted Yields (2025):** [Comment on the spatial distribution of predictions. Do they make sense geographically? Are there unexpected highs/lows? E.g., "The 2025 predictions show higher yields in [specific areas] and lower in [other areas], generally aligning with known coffee growing conditions."]
# * **Feature Importance:** [Highlight the most important features. Do these align with domain knowledge? E.g., "Vegetation indices (NDVI, SAVI) from the preceding months, along with total precipitation and average temperature, were consistently identified as the most impactful features, which aligns with agricultural understanding."]

# **Potential Next Steps:**
# 1.  **Hyperparameter Tuning:** Further optimize the hyperparameters of the chosen best model (e.g., using GridSearchCV or RandomizedSearchCV) to potentially improve performance.
# 2.  **More Advanced Feature Engineering:** Explore more sophisticated temporal aggregations (e.g., cumulative sums of rainfall over specific growing phases, rolling averages) or interaction terms between features.
# 3.  **Alternative Models:** Experiment with other machine learning models (e.g., LightGBM, CatBoost, neural networks) that might capture more complex non-linear relationships.
# 4.  **Data Augmentation:** If more historical yield data becomes available, integrate it to potentially train a more robust model.
# 5.  **Uncertainty Quantification:** Implement methods to quantify the uncertainty of predictions (e.g., using quantile regression forests or Bayesian methods) to provide decision-makers with a range of possible outcomes.
# 6.  **Deployment:** Integrate the trained model into a production environment (e.g., a web application, a data dashboard) to provide real-time or regular yield forecasts.


Sources
